In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from pandas import read_csv

NU_FIDUCIAL = 70.0e6 # 70 MHz
DURATION = 3600. # 3600 seconds
kB = 1.38e-16 # in cgs
cc = 3.e10  # in cgs

# Radio Detectability of Hot Jupiters
For instruments GMRT, HERA, LOFAR, SKA, UTR-2, VLA, we want to estimate Sensitivity, Time, Resolution, Confusion Limit

The radiometer equation:
$$
\frac{\sigma}{1 \rm~Jy} = 10^{23} \frac{2 k}{\lambda^2} \frac{T_{\rm sys} \Omega_{\rm station}}{\sqrt{2 B t N^2/2}}
$$

where $B$ is the bandwidth, $t$ is the observing time, and $N$ is the number of antennas.

or, since 
$$\Omega \approx (\lambda / D)^2$$
we have
$$
\frac{\sigma}{1 \rm~Jy} \approx 
10^{23} \frac{2 k}{D^2} \frac{T_{\rm sys}}{\sqrt{2 B t N^2/2}}
$$

Note that the beam is actually
$$\Omega = \pi (1.22 \lambda / 2 D)^2 / \ln(2)$$
so the equation is
$$
\frac{\sigma}{1 \rm~Jy} = 
10^{23} \frac{2 k \, 1.22^2 \, \pi}{4 \ln{(2)} D^2} \frac{T_{\rm sys}}{\sqrt{2 B t N^2/2}}
$$
which is larger than the approximation by a factor of 1.69.

Galactic synchrotron dominates below 100MHz (Di Matteo+ 2002, Santos, Cooray, Knox 2005, McQuinn+ 2006).

Tsys from Rogers & Bowman (2008)  
Tsys = Tsky = 237K(nu/150MHz)**-2.52

In [2]:
def rms_Jy(observatory, tt, nu=70e6):
    """
    Returns the RMS noise in Jy for an observatory for a certain time and frequency.
    
    Default frequency is 70 MHz.
    """
    nn = observatory['num_antennas']
    num_baselines = nn * (nn - 1.) / 2.
    Tsys = 237 * (nu / 150e6)**(-2.52)
    lam = cc / nu
    bb = observatory['bandwidth_MHz'] * 1.0e6 # convert MHz to Hz
    DD = observatory['dish_diam_m'] * 100. # convert meters to cm
    area_eff = np.pi * (DD/2.)**2
    prefactor = 1e23 * 2. * 1.22**2 * np.pi / (4 *np.log(2)) * kB / area_eff
    return prefactor * Tsys / np.sqrt(2 * bb * tt * num_baselines)

In [3]:
# get table data
observatory_data = read_csv('telescope_data.csv')
observatory_data

,telescope,dish_diam_m,max_baseline_m,num_antennas,bandwidth_MHz
0,gmrt,45.00,25000,30,50
1,hera,14.00,1000,350,50
2,lofar_lba,35.00,100000,44,50
3,mwa,4.00,1000,128,50
4,ska,1.75,65000,130000,50
5,vla,25.00,36000,29,50
6,lwa,2.00,100000,256,50


In [4]:
# Add a column to observatory_data
observatory_data['rms_Jy'] = np.nan * observatory_data.num_antennas
observatory_data

,telescope,dish_diam_m,max_baseline_m,num_antennas,bandwidth_MHz,rms_Jy
0,gmrt,45.00,25000,30,50,NaN
1,hera,14.00,1000,350,50,NaN
2,lofar_lba,35.00,100000,44,50,NaN
3,mwa,4.00,1000,128,50,NaN
4,ska,1.75,65000,130000,50,NaN
5,vla,25.00,36000,29,50,NaN
6,lwa,2.00,100000,256,50,NaN


In [5]:
# We'll do the calculation assuming 70 MHz and 1 hr, with a bandwidth of 100 MHz
# that's a wavelength of 4.3 meters
observatories = {}
for ii, observatory in enumerate(observatory_data.telescope):
    this_observatory = {}
    for this_property in observatory_data:
        if this_property == 'telescope':
            continue
        this_observatory[this_property] = observatory_data[this_property].iloc[ii]
    observatory_data.loc[ii, 'rms_Jy'] = rms_Jy(this_observatory, DURATION, nu=NU_FIDUCIAL)
    observatories[observatory] = this_observatory
observatories

{'gmrt': {'bandwidth_MHz': 50,
  'dish_diam_m': 45.0,
  'max_baseline_m': 25000,
  'num_antennas': 30,
  'rms_Jy': nan},
 'hera': {'bandwidth_MHz': 50,
  'dish_diam_m': 14.0,
  'max_baseline_m': 1000,
  'num_antennas': 350,
  'rms_Jy': nan},
 'lofar_lba': {'bandwidth_MHz': 50,
  'dish_diam_m': 35.0,
  'max_baseline_m': 100000,
  'num_antennas': 44,
  'rms_Jy': nan},
 'lwa': {'bandwidth_MHz': 50,
  'dish_diam_m': 2.0,
  'max_baseline_m': 100000,
  'num_antennas': 256,
  'rms_Jy': nan},
 'mwa': {'bandwidth_MHz': 50,
  'dish_diam_m': 4.0,
  'max_baseline_m': 1000,
  'num_antennas': 128,
  'rms_Jy': nan},
 'ska': {'bandwidth_MHz': 50,
  'dish_diam_m': 1.75,
  'max_baseline_m': 65000,
  'num_antennas': 130000,
  'rms_Jy': nan},
 'vla': {'bandwidth_MHz': 50,
  'dish_diam_m': 25.0,
  'max_baseline_m': 36000,
  'num_antennas': 29,
  'rms_Jy': nan}}

In [6]:
observatory_data

,telescope,dish_diam_m,max_baseline_m,num_antennas,bandwidth_MHz,rms_Jy
0,gmrt,45.00,25000,30,50,0.000378
1,hera,14.00,1000,350,50,0.000330
2,lofar_lba,35.00,100000,44,50,0.000424
3,mwa,4.00,1000,128,50,0.011076
4,ska,1.75,65000,130000,50,0.000057
5,vla,25.00,36000,29,50,0.001269
6,lwa,2.00,100000,256,50,0.022109


In [7]:
# in mJy
observatory_data.rms_Jy * 1000.

0     0.378298
1     0.329849
2     0.424054
3    11.076217
4     0.056755
5     1.268704
6    22.108955
Name: rms_Jy, dtype: float64

# Confusion noise
Want 1 source per 10 beams (pixels)  
Perley & Ericson (1974)  
Wieringa (1991)

$$
\sigma^2 = \int_\Omega \int_0^{5 \sigma / P} s^2 n[s] B[\Omega]^2 P[\Omega]^2 ds d\Omega
$$

$s$: source flux  
$n$: # count at $s$   -- $k s^{-1.65}$ (close to 3/2) ... and $k \propto \nu^{-0.7}$  
$B$: synthesized beam  
$P$: primary beam  
$ds$: sources  
$d\Omega$: solid angle

Remove all sources above 5$\sigma$

At 150 MHz with MWA, for $P \sim 30^\circ$, $B \sim 3.4'$, we get $\sigma \sim 0.1 \rm~Jy$.

$b_{\rm max}$ is the maximum baseline

Should scale as
$$
\sigma^2 \propto \left( \frac{\lambda}{D} \right)^2  \left( \frac{\lambda}{b_{\rm max}} \right)^2 \, ,
$$
or
$$
\sigma \propto \lambda^2 ,
$$

so at 70 MHz should be a factor of 4 worse.

So,

$$
5\sigma \sim 0.1 {\rm~Jy} \times \left( \frac{\nu}{150 \rm~MHz} \right)^{-2} \left( \frac{D}{4 \rm~m} \right)^{-2} \left( \frac{b_{\rm max}}{1000 \rm~m} \right)^{-2}
$$

In the dense regime, the $D$ drops out.  In the sparse regime, the $D$ remains.

$$
\sigma \sim 0.2 {\rm~mJy} \left( \frac{\nu}{1 \rm~GHz} \right)^{-0.7} \left( \frac{\theta}{1'} \right)^2
$$

TODO:  
replace $\theta$ with $\lambda / D$ and express in $\nu$ and $D$, and multiply by 5 for crude estimate of $5\sigma$.  
Wait, should it actually be $\lambda / b_{\rm max}$??  
I'm going to go with that.

\begin{eqnarray}
5 \sigma & \sim {\rm~mJy} \left( \frac{\nu}{1 \rm~GHz} \right)^{-0.7} \left( \frac{\lambda / b_{\rm max}}{1'} \frac{60'}{1^\circ} \frac{180^\circ}{\pi \rm~radians} \right)^2 \\\
 & = 1.18 \times 10^7 {\rm~mJy} \left( \frac{\nu}{1 \rm~GHz} \right)^{-0.7} \left( \frac{\lambda}{b_{\rm max}} \right)^2 \\\
 & = 1.18 \times 10^7 {\rm~mJy} \left( \frac{\nu}{1 \rm~GHz} \right)^{-0.7} \left( \lambda \right)^2 \left( b_{\rm max} \right)^{-2} \\\
 & = 1.18 \times 10^7 {\rm~mJy} \left( \frac{\nu}{1 \rm~GHz} \right)^{-0.7} \left( c / \nu \right)^2 \left( b_{\rm max} \right)^{-2} \\\
 & = 1.06 \times 10^{24} {\rm~mJy} \left( \frac{\nu}{1 \rm~GHz} \right)^{-0.7} {\rm~m^2~s^{-2}} \left( \nu \right)^{-2} \left( b_{\rm max} \right)^{-2} \\\
 & = 1.06 \times 10^{24} {\rm~mJy} \left( \frac{\nu}{1 \rm~GHz} \right)^{-0.7} {\rm~m^2~s^{-2}} \left(10^9 \rm~Hz \right)^{-2} \left( \frac{\nu}{1 \rm~GHz} \right)^{-2} \left( b_{\rm max} \right)^{-2} \\\
 & = 1.06 \times 10^6 {\rm~mJy} \left( \frac{\nu}{1 \rm~GHz} \right)^{-0.7} {\rm~m^2} \left( \frac{\nu}{1 \rm~GHz} \right)^{-2} \left( b_{\rm max} \right)^{-2} \\\
 & = 1.06 \times 10^6 {\rm~mJy} \left( \frac{\nu}{1 \rm~GHz} \right)^{-2.7} {\rm~m^2} ({1000 \rm~m})^{-2} \left( \frac{b_{\rm max}}{1000 \rm~m} \right)^{-2} \\\
 & = 1.06 {\rm~mJy} \left( \frac{\nu}{1 \rm~GHz} \right)^{-2.7} \left( \frac{b_{\rm max}}{1000 \rm~m} \right)^{-2} \\\
 & = 178 {\rm~mJy} \left( \frac{\nu}{150 \rm~MHz} \right)^{-2.7} \left( \frac{b_{\rm max}}{1000 \rm~m} \right)^{-2} \\\
 & = 1400 {\rm~mJy} \left( \frac{\nu}{70 \rm~MHz} \right)^{-2.7} \left( \frac{b_{\rm max}}{1000 \rm~m} \right)^{-2} \\\
\end{eqnarray}

dense limit applies to GMRT?, HERA, LOFAR, MWA, SKA, VLA?, LWA

configuration A for VLA to maximize resolution.

In [8]:
observatory_data['confusion_microJy_per_beam'] = np.nan * observatory_data.num_antennas
observatory_data

,telescope,dish_diam_m,max_baseline_m,num_antennas,bandwidth_MHz,rms_Jy,confusion_microJy_per_beam
0,gmrt,45.00,25000,30,50,0.000378,NaN
1,hera,14.00,1000,350,50,0.000330,NaN
2,lofar_lba,35.00,100000,44,50,0.000424,NaN
3,mwa,4.00,1000,128,50,0.011076,NaN
4,ska,1.75,65000,130000,50,0.000057,NaN
5,vla,25.00,36000,29,50,0.001269,NaN
6,lwa,2.00,100000,256,50,0.022109,NaN


In [9]:
def confusion_microJy(observatory, nu=70e6):
    bmax = observatory['max_baseline_m'] # in m
    answer_mJy = 1393 * (nu / 70e6)**(-2.7) * (bmax / 1000)**(-2)
    answer_microJy = answer_mJy * 1000.
    return answer_microJy

In [10]:
for ii, observatory in enumerate(observatory_data.telescope):
    this_observatory = {}
    for this_property in observatory_data:
        if this_property == 'telescope':
            continue
        this_observatory[this_property] = observatory_data[this_property].iloc[ii]
    observatory_data.loc[ii,
                         'confusion_microJy_per_beam'] = confusion_microJy(this_observatory,
                                                                           nu=NU_FIDUCIAL)
observatory_data

,telescope,dish_diam_m,max_baseline_m,num_antennas,bandwidth_MHz,rms_Jy,confusion_microJy_per_beam
0,gmrt,45.00,25000,30,50,0.000378,2228.800000
1,hera,14.00,1000,350,50,0.000330,1393000.000000
2,lofar_lba,35.00,100000,44,50,0.000424,139.300000
3,mwa,4.00,1000,128,50,0.011076,1393000.000000
4,ska,1.75,65000,130000,50,0.000057,329.704142
5,vla,25.00,36000,29,50,0.001269,1074.845679
6,lwa,2.00,100000,256,50,0.022109,139.300000
